In [ ]:
# Dataset download 

# !git lfs install
# !git clone https://github.com/babylonhealth/primock57.git
# !cd scripts
# !python extract_utterances.py --audio_path=../audio --transcript_path=../transcripts --output_path=../output

In [1]:
# setup env

# !conda create --name clinical_asr python==3.10.12
# !conda activate clinical_asr
# !pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

# %pip install -U openai-whisper
# !pip install nemo_toolkit[all]

In [1]:
import os
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import glob
import soundfile as sf
from whisper.normalizers import EnglishTextNormalizer
from jiwer import wer
import jiwer
import whisper
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
normalizer = EnglishTextNormalizer()

transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveWhiteSpace(replace_by_space=True),
    jiwer.RemoveMultipleSpaces(),
    jiwer.ReduceToListOfListOfWords(word_delimiter=" ")
    ]) 

/home2/neha.sherin/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# calc  and print WER

def print_wer(references, hypotheses):
    
    data = pd.DataFrame(dict(hypothesis=hypotheses, reference=references)) #, phonemes=gtphonms))
    print("WER:", wer(list(data["reference"]), list(data["hypothesis"]), truth_transform=transformation, hypothesis_transform=transformation ))

    for hyp, ref in zip(data["hypothesis"],data["reference"]):
        if hyp!="" and ref!="":
            data["hypothesis_clean"] = normalizer(hyp)
            data["reference_clean"] = normalizer(ref)                              
    print("WER without fillers:", wer(list(data["reference_clean"]), list(data["hypothesis_clean"]), truth_transform=transformation, hypothesis_transform=transformation ))

    

In [3]:
# dataset class

audio_path = "primock57/output/audio_utterances/"
transcripts_file = "primock57/output/test_transcript.ref.txt"

class ClinDataset(torch.utils.data.Dataset):

    def __init__(self, device=DEVICE):

        self.device = device
        
        self.transcripts = {}
        with open(transcripts_file,'r') as vf:
            trans = vf.readlines()
        for t in trans:
            self.transcripts[t.split('|')[1].strip()] = t.split('|')[0]  
        
        self.wavpaths = []
        for wp in os.listdir(audio_path):
            basename = wp[:-4]
            try:
                t = self.transcripts[basename]
                self.wavpaths.append(wp)
            except:
                pass
        
        print('num files',len(self.wavpaths))
        
               
    def __len__(self):
        return len(self.wavpaths)

    def __getitem__(self, idx):

        wav_path = os.path.join(audio_path, self.wavpaths[idx])
        audio, _ = sf.read(wav_path)
        
        audio_wh = whisper.load_audio(wav_path)
        audio_wh = whisper.pad_or_trim(audio_wh)
        mel = whisper.log_mel_spectrogram(audio_wh).to(self.device)
        
        basename = wav_path.split('/')[-1][:-4]
        transcript = self.transcripts[basename]
        if transcript=="":
            print(basename)

        return transcript, audio, mel, wav_path

dataset = ClinDataset()
loader = torch.utils.data.DataLoader(dataset, batch_size=1)

num files 671


In [5]:
# 1. Wav2Vec - pretrained

In [6]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
# from datasets import load_dataset

# load model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")


hypotheses = []
references = []
for txts, wavs, mel, _ in tqdm(loader):

    # tokenize
    input_values = tokenizer(wavs, return_tensors="pt", padding="longest").input_values  # Batch size 1
    input_values = input_values.squeeze(0)
    # retrieve logits
    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)
    hypotheses.extend(transcription)
    references.extend(txts)

print_wer(references, hypotheses)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/home2/neha.sherin/miniconda3/envs/nemo/lib/python3.10/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:733: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a Bert

WER: 0.42919854508967764
WER without fillers: 0.6666666666666666


In [7]:
# 2. whisper asr

In [8]:
model = whisper.load_model("small.en")
print(
    f"Model is English-only "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)
# predict without timestamps for short-form transcription
options = whisper.DecodingOptions(language="en", without_timestamps=True)

hypotheses = []
references = []

for texts, wavs, mels, _ in tqdm(loader):
    if texts=="":
        print(basename)
    results = model.decode(mels, options)
    hypotheses.extend([result.text for result in results])
    references.extend(texts)
    
print_wer(references, hypotheses)

Model is English-only and has 240,582,144 parameters.


100%|███████████████████████████████████████████████████████████████| 671/671 [04:40<00:00,  2.39it/s]

WER: 0.34378527530415154
WER without fillers: 0.16666666666666666


In [9]:
# 3. NeMo ASR (Fastconformer Transducer Large)

In [10]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.ASRModel.from_pretrained("stt_en_fastconformer_transducer_large")


hypotheses = []
references = []

for texts, wavs, mels, wavpath in tqdm(loader):
    results = asr_model.transcribe([wavpath])[0]
    hypotheses.extend(results)
    references.extend(texts)

print_wer(references, hypotheses)

[NeMo I 2024-01-14 03:14:26 cloud:58] Found existing object /home2/neha.sherin/.cache/torch/NeMo/NeMo_1.22.0/stt_en_fastconformer_transducer_large/74b4e49a2b22465da737dd5345767e61/stt_en_fastconformer_transducer_large.nemo.
[NeMo I 2024-01-14 03:14:26 cloud:64] Re-using file from: /home2/neha.sherin/.cache/torch/NeMo/NeMo_1.22.0/stt_en_fastconformer_transducer_large/74b4e49a2b22465da737dd5345767e61/stt_en_fastconformer_transducer_large.nemo
[NeMo I 2024-01-14 03:14:26 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2024-01-14 03:14:26 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-01-14 03:14:27 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-01-14 03:14:27 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    num_workers: 8
    pin_m

[NeMo I 2024-01-14 03:14:27 features:289] PADDING: 0


[NeMo W 2024-01-14 03:14:28 nemo_logging:349] /home2/neha.sherin/miniconda3/envs/nemo/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2024-01-14 03:14:28 rnnt_models:211] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-01-14 03:14:28 audio_preprocessing:517] Numba CUDA SpecAugment kernel is being used
[NeMo I 2024-01-14 03:14:29 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /home2/neha.sherin/.cache/torch/NeMo/NeMo_1.22.0/stt_en_fastconformer_transducer_large/74b4e49a2b22465da737dd5345767e61/stt_en_fastconformer_transducer_large.nemo.


...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████▉| 670/671 [02:04<00:00,  5.06it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████| 671/671 [02:04<00:00,  5.40it/s]


WER: 0.16932145992725448
WER without fillers: 0.5


In [ ]:
print_wer(references, hypotheses)

In [ ]:
# finetune wav2vec model

from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

for texts, wavs, mels, wavpath in tqdm(loader):

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:
    processor: AutoProcessor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"][0]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

        labels_batch = self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch
    
data_collator = DataCollatorCTCWithPadding(processor=processor, padding="longest")